# Initial Data Import and Clean Up

Importing 4 csv files: 
    1. "macro_small" has some of the macroeconomic data that I thought would be the most relevant, and some simple yearly standard deviations as a measure of volatility: timestamp, oil_urals, cpi, ppi, eurrub, brent, average_provision_of_build_contract_moscow, micex, mortgage_rate, rent_price_4+room_bus, sd_oil_yearly, sd_cpi_yearly, sd_ppi_yearly, sd_eurrub_yearly, sd_brent_yearly, sd_micex_yearly, sd_mortgage_yearly, sd_rent_yearly
    2. "train_small" has all columns from the training data set expect for the distances to features (like *..._500...*)
    3. "test_small" has the same columns as the training set
    4. "okurg" has the identifying region for the sub regions

In [3]:
#Import modules:
import pandas as pd
import numpy as np
import scipy as sp
import sklearn

In [4]:
#import data and merge the macro onto the train and test
trainsm_df = pd.read_csv("train_small.csv", parse_dates=['timestamp'])
testsm_df = pd.read_csv("test_small.csv", parse_dates=['timestamp'])
macrosm_df = pd.read_csv("macro_small.csv", parse_dates=['timestamp'])
trainsm_df = pd.merge(trainsm_df, macrosm_df, how='left', on='timestamp')
testsm_df = pd.merge(testsm_df, macrosm_df, how='left', on='timestamp')
print(trainsm_df.shape, testsm_df.shape)

#truncate the extreme values in price_doc
ulimit = np.percentile(trainsm_df.price_doc.values, 99)
llimit = np.percentile(trainsm_df.price_doc.values, 1)
trainsm_df['price_doc'].loc[trainsm_df['price_doc']>ulimit] = ulimit
trainsm_df['price_doc'].loc[trainsm_df['price_doc']<llimit] = llimit

((30471, 132), (7662, 131))


/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
#attach the okrug region
okurg_df = pd.read_csv("okurg.csv")
trainsm_df = pd.merge(trainsm_df, okurg_df, how='left', on='sub_area')
testsm_df = pd.merge(testsm_df, okurg_df, how='left', on='sub_area')

In [5]:
#Checking out the data
dtype_df = trainsm_df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df.groupby("Column Type").aggregate('count').reset_index()

,Column Type,Count
0,int64,22
1,datetime64[ns],1
2,float64,107
3,object,3


In [6]:
dtype_df2 = testsm_df.dtypes.reset_index()
dtype_df2.columns = ["Count", "Column Type"]
dtype_df2.groupby("Column Type").aggregate('count').reset_index()

,Column Type,Count
0,int64,25
1,datetime64[ns],1
2,float64,103
3,object,3


Convert the objects so that we can manipulate them using regression

In [7]:
from sklearn import model_selection, preprocessing
for f in trainsm_df.columns:
    if trainsm_df[f].dtype=='object':
        print(f)
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(trainsm_df[f].values.astype('str')) + list(testsm_df[f].values.astype('str')))
        trainsm_df[f] = lbl.transform(list(trainsm_df[f].values.astype('str')))
        testsm_df[f] = lbl.transform(list(testsm_df[f].values.astype('str')))

product_type
sub_area
okurg_district


In [8]:
missing = trainsm_df.isnull().sum()
print(missing.to_string())
print len(trainsm_df)

id                                                0
timestamp                                         0
full_sq                                           0
life_sq                                        6383
floor                                           167
max_floor                                      9572
material                                       9572
build_year                                    13605
num_room                                       9572
kitch_sq                                       9572
state                                         13559
product_type                                      0
sub_area                                          0
area_m                                            0
raion_popul                                       0
green_zone_part                                   0
indust_part                                       0
children_preschool                                0
preschool_quota                                6688
preschool_ed

In [10]:
trainsm_df['year'] = pd.DatetimeIndex(trainsm_df['timestamp']).year

In [11]:
trainsm_df.describe()

,id,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,rent_price_4+room_bus,sd_oil_yearly,sd_cpi_yearly,sd_ppi_yearly,sd_eurrub_yearly,sd_brent_yearly,sd_micex_yearly,sd_mortgage_yearly,sd_rent_yearly,year
count,30471.000000,30471.000000,24088.000000,30304.000000,20899.000000,20899.000000,1.686600e+04,20899.000000,20899.000000,16912.000000,...,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000
mean,15237.917397,54.214269,34.403271,7.670803,12.558974,1.827121,3.068057e+03,1.909804,6.399301,2.107025,...,163.273194,9.646020,9.260170,12.611478,4.319069,9.947686,71.534509,0.291013,16.956112,2013.452726
std,8796.501536,38.031487,52.285733,5.319989,6.756550,1.481154,1.543878e+05,0.851805,28.265979,0.880148,...,20.473868,4.575543,2.474582,4.315761,2.660853,4.439635,12.171822,0.149343,5.012870,0.962433
min,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000e+00,0.000000,0.000000,1.000000,...,100.080000,3.808905,5.080692,9.982029,0.767611,4.262096,67.400112,0.220400,10.567959,2011.000000
25%,7620.500000,38.000000,20.000000,3.000000,9.000000,1.000000,1.967000e+03,1.000000,1.000000,1.000000,...,149.100000,3.808905,6.802652,9.982029,1.769683,4.262096,67.400112,0.220400,10.567959,2013.000000
50%,15238.000000,49.000000,30.000000,6.500000,12.000000,1.000000,1.979000e+03,2.000000,6.000000,2.000000,...,160.690000,8.009779,10.395346,10.704748,6.599686,8.039559,68.573449,0.248999,16.823747,2014.000000
75%,22855.500000,63.000000,43.000000,11.000000,17.000000,2.000000,2.005000e+03,2.000000,9.000000,3.000000,...,174.220000,14.488195,10.395346,14.312976,6.599686,14.648207,71.295861,0.257487,20.825967,2014.000000
max,30473.000000,5326.000000,7478.000000,77.000000,117.000000,6.000000,2.005201e+07,19.000000,2014.000000,33.000000,...,212.870000,14.488195,14.482090,24.372704,7.036363,14.648207,146.135335,0.717897,27.602766,2015.000000


In [6]:
missing2 = testsm_df.isnull().sum()
print(missing2.to_string())
print len(testsm_df)

id                                               0
timestamp                                        0
full_sq                                          0
life_sq                                       1176
floor                                            0
max_floor                                        0
material                                         0
build_year                                    1049
num_room                                         0
kitch_sq                                         0
state                                          694
product_type                                    33
sub_area                                         0
area_m                                           0
raion_popul                                      0
green_zone_part                                  0
indust_part                                      0
children_preschool                               0
preschool_quota                               1596
preschool_education_centers_rai

Get rid of a few of the observations with missing values from the training set: floor (only 130 missing), and walk to the metro (only 25 missing). Walk to metro is also missing in Test (only 34). 

In [21]:
#fix train
train_nofloor = trainsm_df.dropna(subset = ['floor'])
train_clean = train_nofloor.dropna(subset = ['metro_min_walk'])

In [7]:
testsm_df.describe()

,id,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,mortgage_rate,rent_price_4+room_bus,sd_oil_yearly,sd_cpi_yearly,sd_ppi_yearly,sd_eurrub_yearly,sd_brent_yearly,sd_micex_yearly,sd_mortgage_yearly,sd_rent_yearly
count,7662.000000,7662.000000,6486.000000,7662.000000,7662.000000,7662.000000,6613.000000,7662.000000,7662.000000,6968.000000,...,7662.000000,7662.000000,7662.000000,7662.000000,7662.000000,7662.000000,7662.000000,7662.000000,7662.000000,7662.000000
mean,34304.500000,53.701164,32.660037,7.652571,12.591099,1.854216,1820.888099,1.876403,6.938662,1.985792,...,12.726691,181.616325,6.960293,10.357489,21.469072,6.039786,7.231745,81.374219,0.484803,29.842628
std,2211.973214,20.104320,22.737855,5.099418,6.659291,1.517171,552.993964,0.835665,25.579688,0.820086,...,0.394161,28.196758,1.008706,3.964332,2.790806,0.957853,0.776425,9.686745,0.224037,2.152828
min,30474.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,...,12.100000,142.850000,5.990909,6.547696,18.787058,5.119272,6.485588,71.295861,0.269500,27.602766
25%,32389.250000,39.200000,19.900000,4.000000,9.000000,1.000000,1966.000000,1.000000,1.000000,1.000000,...,12.290000,164.260000,5.990909,6.547696,18.787058,5.119272,6.485588,71.295861,0.269500,27.602766
50%,34304.500000,50.410000,30.400000,7.000000,14.000000,1.000000,1982.000000,2.000000,7.000000,2.000000,...,12.860000,174.650000,5.990909,6.547696,18.787058,5.119272,6.485588,90.683353,0.269500,31.911534
75%,36219.750000,63.300000,41.800000,11.000000,17.000000,2.000000,2011.000000,2.000000,9.500000,3.000000,...,12.920000,205.640000,8.009779,14.482090,24.372704,7.036363,8.039559,90.683353,0.717897,31.911534
max,38135.000000,403.000000,869.000000,41.000000,48.000000,6.000000,2019.000000,19.000000,1968.000000,4.000000,...,13.440000,242.230000,8.009779,14.482090,24.372704,7.036363,8.039559,90.683353,0.717897,31.911534


In [9]:
testsm_df.loc[testsm_df['max_floor'] < testsm_df['floor']]

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,rent_price_4+room_bus,sd_oil_yearly,sd_cpi_yearly,sd_ppi_yearly,sd_eurrub_yearly,sd_brent_yearly,sd_micex_yearly,sd_mortgage_yearly,sd_rent_yearly,okurg_district
5,30479,2015-07-01,48.43,NaN,21,1,1,2015.0,1,1.00,...,211.83,8.009779,14.482090,24.372704,7.036363,8.039559,71.295861,0.717897,27.602766,Western
7,30481,2015-07-01,43.10,NaN,5,0,1,NaN,1,0.00,...,211.83,8.009779,14.482090,24.372704,7.036363,8.039559,71.295861,0.717897,27.602766,Novomoskovsky
14,30488,2015-07-01,69.30,NaN,7,1,1,NaN,2,1.00,...,211.83,8.009779,14.482090,24.372704,7.036363,8.039559,71.295861,0.717897,27.602766,Novomoskovsky
15,30489,2015-07-02,69.30,NaN,10,0,1,NaN,2,0.00,...,211.83,8.009779,14.482090,24.372704,7.036363,8.039559,71.295861,0.717897,27.602766,Novomoskovsky
16,30490,2015-07-02,117.40,NaN,10,0,1,NaN,3,0.00,...,211.83,8.009779,14.482090,24.372704,7.036363,8.039559,71.295861,0.717897,27.602766,Eastern
32,30506,2015-07-02,53.82,NaN,3,1,1,NaN,2,1.00,...,211.83,8.009779,14.482090,24.372704,7.036363,8.039559,71.295861,0.717897,27.602766,Troitsky
44,30518,2015-07-03,35.82,1.00,14,1,1,2015.0,1,1.00,...,211.83,8.009779,14.482090,24.372704,7.036363,8.039559,71.295861,0.717897,27.602766,Western
68,30542,2015-07-06,34.80,NaN,3,0,1,NaN,1,0.00,...,211.83,8.009779,14.482090,24.372704,7.036363,8.039559,71.295861,0.717897,27.602766,Troitsky
72,30546,2015-07-06,41.90,41.90,9,0,1,0.0,1,1.00,...,211.83,8.009779,14.482090,24.372704,7.036363,8.039559,71.295861,0.717897,27.602766,Novomoskovsky
73,30547,2015-07-06,61.70,NaN,7,1,1,2015.0,2,1.00,...,211.83,8.009779,14.482090,24.372704,7.036363,8.039559,71.295861,0.717897,27.602766,North Eastern


In [22]:
#fix test
test_clean = testsm_df.dropna(subset = ['metro_min_walk'])

Drop columns that have missing values in both the test and train datasets

In [24]:
#Make a new column that is just the sq_mter, to get rid of issues with Life Square not existing or being incorrect
train_clean.loc[:, 'sq_metr'] = train_nofloor.loc[:, ['full_sq','life_sq']].max(axis=1)
test_clean.loc[:, 'sq_metr'] = test_clean.loc[:, ['full_sq','life_sq']].max(axis=1)

In [27]:
train_clean = train_clean.dropna(axis=1, how='any')
test_clean = test_clean.dropna(axis=1, how='any')
test_clean = test_clean.drop(["max_floor", "material", "num_room", "kitch_sq"], axis=1)

# Remove or Fix unusual observations:

In [28]:
train_clean.describe()

,id,full_sq,floor,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,...,sd_oil_yearly,sd_cpi_yearly,sd_ppi_yearly,sd_eurrub_yearly,sd_brent_yearly,sd_micex_yearly,sd_mortgage_yearly,sd_rent_yearly,okurg_district,sq_metr
count,30279.000000,30279.000000,30279.000000,30279.000000,30279.000000,3.027900e+04,30279.000000,30279.000000,30279.000000,30279.000000,...,30279.000000,30279.000000,30279.000000,30279.000000,30279.000000,30279.000000,30279.000000,30279.000000,30279.000000,30279.000000
mean,15285.384821,54.145249,7.670597,0.360745,76.412596,1.763337e+07,84180.148816,0.218657,0.119070,5147.974834,...,9.667762,9.271400,12.611831,4.334090,9.968345,71.509127,0.291085,16.961970,5.233396,54.516563
std,8787.583362,38.049660,5.319877,0.480225,39.233146,2.057831e+07,57891.428401,0.175108,0.118731,3818.824337,...,4.576251,2.473901,4.321397,2.658538,4.440540,12.116159,0.149596,5.010406,3.094380,57.537209
min,1.000000,0.000000,0.000000,0.000000,0.000000,2.081628e+06,2546.000000,0.001879,0.000000,175.000000,...,3.808905,5.080692,9.982029,0.767611,4.262096,67.400112,0.220400,10.567959,0.000000,0.000000
25%,7698.000000,38.000000,3.000000,0.000000,44.000000,7.307411e+06,21819.000000,0.063755,0.019509,1706.000000,...,3.808905,6.802652,9.982029,1.769683,4.262096,67.400112,0.220400,10.567959,3.000000,38.000000
50%,15313.000000,49.000000,7.000000,0.000000,77.000000,1.041657e+07,83502.000000,0.167526,0.072158,4926.000000,...,8.009779,10.395346,10.704748,6.599686,8.039559,68.573449,0.248999,16.823747,5.000000,49.000000
75%,22890.500000,63.000000,11.000000,1.000000,105.000000,1.803644e+07,122862.000000,0.336177,0.195781,7103.000000,...,14.488195,10.395346,14.312976,6.599686,14.648207,71.295861,0.257487,20.825967,7.000000,63.000000
max,30473.000000,5326.000000,77.000000,1.000000,145.000000,2.060718e+08,247469.000000,0.852923,0.521867,19223.000000,...,14.488195,14.482090,24.372704,7.036363,14.648207,146.135335,0.717897,27.602766,12.000000,7478.000000


In [31]:
#find where the meters does not make sense (20 instances)
train_clean.loc[train_clean['sq_metr'] < 6]
train_clean = train_clean.drop(train_clean[train_clean['sq_metr'] < 6].index)

In [29]:
test_clean.describe()

,id,full_sq,floor,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,...,sd_oil_yearly,sd_cpi_yearly,sd_ppi_yearly,sd_eurrub_yearly,sd_brent_yearly,sd_micex_yearly,sd_mortgage_yearly,sd_rent_yearly,okurg_district,sq_metr
count,7628.000000,7628.00000,7628.000000,7628.000000,7628.000000,7.628000e+03,7628.000000,7628.000000,7628.000000,7628.000000,...,7628.000000,7628.000000,7628.000000,7628.000000,7628.000000,7628.000000,7628.000000,7628.000000,7628.000000,7628.000000
mean,34302.871919,53.65292,7.627819,0.352255,77.825773,1.771468e+07,87567.132407,0.224865,0.122595,5358.472208,...,6.961703,10.363029,21.472972,6.041124,7.232830,81.360684,0.485116,29.839620,5.370608,53.960302
std,2211.414306,20.08461,5.072910,0.486677,40.165773,2.215143e+07,56419.270784,0.177155,0.126425,3776.316459,...,1.008761,3.964550,2.790959,0.957906,0.776467,9.687276,0.224049,2.152946,3.225320,23.988084
min,30474.000000,0.00000,1.000000,0.000000,0.000000,2.081628e+06,2546.000000,0.001879,0.000000,175.000000,...,5.990909,6.547696,18.787058,5.119272,6.485588,71.295861,0.269500,27.602766,0.000000,2.000000
25%,32387.750000,39.20000,3.000000,0.000000,44.000000,7.307411e+06,37502.000000,0.068202,0.012339,2448.000000,...,5.990909,6.547696,18.787058,5.119272,6.485588,71.295861,0.269500,27.602766,3.000000,39.200000
50%,34300.500000,50.30000,7.000000,0.000000,82.000000,1.004686e+07,85956.000000,0.157332,0.072158,4992.000000,...,5.990909,6.547696,18.787058,5.119272,6.485588,90.683353,0.269500,31.911534,5.000000,50.350000
75%,36220.250000,63.20000,11.000000,1.000000,107.000000,1.788191e+07,118945.000000,0.349935,0.195781,7125.000000,...,8.009779,14.482090,24.372704,7.036363,8.039559,90.683353,0.717897,31.911534,8.000000,63.300000
max,38135.000000,403.00000,41.000000,2.000000,145.000000,2.060718e+08,247469.000000,0.852923,0.521867,19223.000000,...,8.009779,14.482090,24.372704,7.036363,8.039559,90.683353,0.717897,31.911534,12.000000,869.000000


In [33]:
#drop one observation in the test as well
test_clean.loc[test_clean['sq_metr'] < 6]
test_clean = test_clean.drop(test_clean[test_clean['sq_metr'] < 6].index)

Using some of Shu's work, cleaning up when the sq_metr doesn't make sense

In [35]:
#when is sq meter too large?
train_clean.loc[train_clean['sq_metr'] > 300]

,id,timestamp,full_sq,floor,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,...,sd_oil_yearly,sd_cpi_yearly,sd_ppi_yearly,sd_eurrub_yearly,sd_brent_yearly,sd_micex_yearly,sd_mortgage_yearly,sd_rent_yearly,okurg_district,sq_metr
130,129,2011-10-15,325,7.0,0,33,1.020722e+07,122862,0.512707,0.000170,...,6.269592,5.080692,13.540192,1.085887,7.113732,146.135335,0.361158,16.374863,1,325.0
1484,1481,2012-03-11,353,5.0,0,124,1.132409e+07,113897,0.083810,0.371149,...,7.219266,7.261960,14.312976,0.767611,7.768621,75.261889,0.248999,20.825967,11,353.0
1622,1613,2012-03-21,394,11.0,0,56,1.788191e+07,165727,0.260653,0.133215,...,7.219266,7.261960,14.312976,0.767611,7.768621,75.261889,0.248999,20.825967,6,394.0
2432,2428,2012-06-07,412,6.0,0,24,9.629358e+06,89467,0.343754,0.238617,...,7.219266,7.261960,14.312976,0.767611,7.768621,75.261889,0.248999,20.825967,11,412.0
2764,2783,2012-07-06,729,12.0,0,133,1.127507e+07,112804,0.331076,0.000991,...,7.219266,7.261960,14.312976,0.767611,7.768621,75.261889,0.248999,20.825967,11,729.0
3522,3530,2012-09-07,5326,13.0,1,10,1.479557e+07,145088,0.308057,0.050900,...,7.219266,7.261960,14.312976,0.767611,7.768621,75.261889,0.248999,20.825967,8,5326.0
3598,3599,2012-09-12,388,9.0,1,72,1.139168e+07,19940,0.055644,0.243205,...,7.219266,7.261960,14.312976,0.767611,7.768621,75.261889,0.248999,20.825967,6,388.0
4376,4388,2012-10-25,73,17.0,1,137,5.333221e+06,64931,0.074077,0.169091,...,7.219266,7.261960,14.312976,0.767611,7.768621,75.261889,0.248999,20.825967,0,426.0
5945,5947,2013-02-07,634,3.0,0,55,5.646405e+06,79576,0.258663,0.101872,...,3.808905,6.802652,9.982029,1.769683,4.262096,67.400112,0.257487,10.567959,2,634.0
7202,7210,2013-04-10,461,2.0,0,141,1.509487e+07,219609,0.300284,0.033494,...,3.808905,6.802652,9.982029,1.769683,4.262096,67.400112,0.257487,10.567959,6,461.0


In [36]:
#Imputation to make the square footage make sense
train_clean.loc[train_clean['sq_metr'] > 1000, 'sq_metr'] = train_clean.loc[train_clean['sq_metr'] > 1000, 'sq_metr']/100
train_clean.loc[train_clean['sq_metr'] > 310, 'sq_metr'] = train_clean.loc[train_clean['sq_metr'] > 310, 'sq_metr']/10

In [37]:
#checking it:
train_clean.loc[train_clean['sq_metr'] > 300]

#the two (9230 and 16727) are very large units that were very expensive

,id,timestamp,full_sq,floor,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,...,sd_oil_yearly,sd_cpi_yearly,sd_ppi_yearly,sd_eurrub_yearly,sd_brent_yearly,sd_micex_yearly,sd_mortgage_yearly,sd_rent_yearly,okurg_district,sq_metr
9230,9240,2013-07-16,47,4.0,0,11,8464344.053,85721,0.075206,0.521867,...,3.808905,6.802652,9.982029,1.769683,4.262096,67.400112,0.257487,10.567959,8,301.0
16727,16746,2014-03-31,303,32.0,1,77,6050064.566,78616,0.167526,0.093443,...,14.488195,10.395346,10.704748,6.599686,14.648207,68.573449,0.220400,16.823747,7,303.0


In [38]:
#when is sq meter too large for the test data?
test_clean.loc[test_clean['sq_metr'] > 300]
#looking at the full_sq for the first three, just looks like they were shifted 1 decmimal -- divide by 10

,id,timestamp,full_sq,floor,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,...,sd_oil_yearly,sd_cpi_yearly,sd_ppi_yearly,sd_eurrub_yearly,sd_brent_yearly,sd_micex_yearly,sd_mortgage_yearly,sd_rent_yearly,okurg_district,sq_metr
601,31075,2015-08-17,74.2,2,1,102,6.677245e+07,9553,0.336177,0.072158,...,8.009779,14.482090,24.372704,7.036363,8.039559,71.295861,0.717897,27.602766,5,742.0
1896,32370,2015-10-23,36.1,14,1,102,6.677245e+07,9553,0.336177,0.072158,...,8.009779,14.482090,24.372704,7.036363,8.039559,71.295861,0.717897,27.602766,5,361.0
2791,33265,2015-12-01,86.9,3,1,18,9.282439e+06,143661,0.153279,0.127376,...,8.009779,14.482090,24.372704,7.036363,8.039559,71.295861,0.717897,27.602766,8,869.0
4196,34670,2016-01-25,314.5,3,0,132,1.631523e+07,39873,0.375974,0.075424,...,5.990909,6.547696,18.787058,5.119272,6.485588,90.683353,0.269500,31.911534,10,314.5
4634,35108,2016-02-02,403.0,7,1,6,8.398461e+06,108171,0.015234,0.037316,...,5.990909,6.547696,18.787058,5.119272,6.485588,90.683353,0.269500,31.911534,0,403.0


In [39]:
test_clean.loc[test_clean['sq_metr'] > 310, 'sq_metr'] = test_clean.loc[test_clean['sq_metr'] > 310, 'sq_metr']/10

In [40]:
test_clean.loc[test_clean['sq_metr'] > 300]

,id,timestamp,full_sq,floor,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,...,sd_oil_yearly,sd_cpi_yearly,sd_ppi_yearly,sd_eurrub_yearly,sd_brent_yearly,sd_micex_yearly,sd_mortgage_yearly,sd_rent_yearly,okurg_district,sq_metr


In [41]:
test_clean.describe()

,id,full_sq,floor,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,...,sd_oil_yearly,sd_cpi_yearly,sd_ppi_yearly,sd_eurrub_yearly,sd_brent_yearly,sd_micex_yearly,sd_mortgage_yearly,sd_rent_yearly,okurg_district,sq_metr
count,7627.000000,7627.000000,7627.000000,7627.000000,7627.000000,7.627000e+03,7627.000000,7627.000000,7627.000000,7627.000000,...,7627.000000,7627.000000,7627.000000,7627.000000,7627.000000,7627.000000,7627.000000,7627.000000,7627.000000,7627.000000
mean,34302.541366,53.659692,7.628163,0.352170,77.818408,1.771604e+07,87568.730694,0.224886,0.122611,5358.619247,...,6.961830,10.363529,21.473324,6.041245,7.232928,81.359461,0.485145,29.839348,5.371312,53.649748
std,2211.370824,20.077214,5.073153,0.486653,40.163254,2.215256e+07,56422.797102,0.177158,0.126425,3776.542208,...,1.008766,3.964569,2.790972,0.957910,0.776471,9.687323,0.224050,2.152956,3.224945,19.722009
min,30474.000000,0.000000,1.000000,0.000000,0.000000,2.081628e+06,2546.000000,0.001879,0.000000,175.000000,...,5.990909,6.547696,18.787058,5.119272,6.485588,71.295861,0.269500,27.602766,0.000000,10.000000
25%,32387.500000,39.200000,3.000000,0.000000,44.000000,7.307411e+06,37502.000000,0.068202,0.012339,2448.000000,...,5.990909,6.547696,18.787058,5.119272,6.485588,71.295861,0.269500,27.602766,3.000000,39.200000
50%,34300.000000,50.300000,7.000000,0.000000,82.000000,1.004686e+07,85956.000000,0.157332,0.072158,4992.000000,...,5.990909,6.547696,18.787058,5.119272,6.485588,90.683353,0.269500,31.911534,5.000000,50.300000
75%,36219.500000,63.200000,11.000000,1.000000,107.000000,1.788191e+07,118945.000000,0.349935,0.195781,7125.000000,...,8.009779,14.482090,24.372704,7.036363,8.039559,90.683353,0.717897,31.911534,8.000000,63.235000
max,38135.000000,403.000000,41.000000,2.000000,145.000000,2.060718e+08,247469.000000,0.852923,0.521867,19223.000000,...,8.009779,14.482090,24.372704,7.036363,8.039559,90.683353,0.717897,31.911534,12.000000,285.800000


In [42]:
train_clean.describe()

,id,full_sq,floor,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,...,sd_oil_yearly,sd_cpi_yearly,sd_ppi_yearly,sd_eurrub_yearly,sd_brent_yearly,sd_micex_yearly,sd_mortgage_yearly,sd_rent_yearly,okurg_district,sq_metr
count,30258.000000,30258.000000,30258.000000,30258.000000,30258.000000,3.025800e+04,30258.000000,30258.000000,30258.000000,30258.000000,...,30258.000000,30258.000000,30258.000000,30258.000000,30258.000000,30258.000000,30258.000000,30258.000000,30258.000000,30258.000000
mean,15280.163758,54.182167,7.673111,0.360301,76.407562,1.763295e+07,84203.903001,0.218667,0.119082,5149.306497,...,9.664416,9.270620,12.613154,4.332518,9.965097,71.511164,0.291134,16.962066,5.233459,53.883559
std,8787.941263,38.037038,5.317614,0.480096,39.241942,2.058212e+07,57889.274079,0.175125,0.118748,3818.782073,...,4.576075,2.474583,4.322604,2.658790,4.440368,12.120116,0.149636,5.012143,3.095044,21.207045
min,1.000000,0.000000,0.000000,0.000000,0.000000,2.081628e+06,2546.000000,0.001879,0.000000,175.000000,...,3.808905,5.080692,9.982029,0.767611,4.262096,67.400112,0.220400,10.567959,0.000000,6.000000
25%,7692.250000,38.000000,3.000000,0.000000,44.000000,7.307411e+06,21819.000000,0.063755,0.019509,1706.000000,...,3.808905,6.802652,9.982029,1.769683,4.262096,67.400112,0.220400,10.567959,3.000000,38.000000
50%,15302.500000,49.000000,7.000000,0.000000,77.000000,1.041657e+07,83502.000000,0.167526,0.072158,4926.000000,...,8.009779,10.395346,10.704748,6.599686,8.039559,68.573449,0.248999,16.823747,5.000000,49.000000
75%,22883.750000,63.000000,11.000000,1.000000,105.000000,1.803644e+07,122862.000000,0.336177,0.195781,7103.000000,...,14.488195,10.395346,14.312976,6.599686,14.648207,71.295861,0.257487,20.825967,7.000000,63.000000
max,30473.000000,5326.000000,77.000000,1.000000,145.000000,2.060718e+08,247469.000000,0.852923,0.521867,19223.000000,...,14.488195,14.482090,24.372704,7.036363,14.648207,146.135335,0.717897,27.602766,12.000000,303.000000


# Do some feature engineering to combine some of the columns of information

Want to create a few different features that help to describe the observations a bit more, and see if those features can add value to the model. I want to inlcude some features that are related to the *population density, gender equality in the workforce, old vs young residents, the log of the price (to make the distribution better?), etc.*

In [43]:
list(train_clean)

['id',
 'timestamp',
 'full_sq',
 'floor',
 'product_type',
 'sub_area',
 'area_m',
 'raion_popul',
 'green_zone_part',
 'indust_part',
 'children_preschool',
 'preschool_education_centers_raion',
 'children_school',
 'school_education_centers_raion',
 'school_education_centers_top_20_raion',
 'healthcare_centers_raion',
 'university_top_20_raion',
 'sport_objects_raion',
 'additional_education_raion',
 'shopping_centers_raion',
 'office_raion',
 'full_all',
 'male_f',
 'female_f',
 'young_all',
 'work_all',
 'work_male',
 'work_female',
 'ekder_all',
 'metro_min_avto',
 'metro_km_avto',
 'metro_min_walk',
 'metro_km_walk',
 'kindergarten_km',
 'school_km',
 'park_km',
 'green_zone_km',
 'industrial_km',
 'water_treatment_km',
 'cemetery_km',
 'incineration_km',
 'railroad_station_walk_km',
 'railroad_station_walk_min',
 'railroad_station_avto_km',
 'railroad_station_avto_min',
 'public_transport_station_km',
 'public_transport_station_min_walk',
 'water_km',
 'mkad_km',
 'ttk_km',
 's

Which features should we keep, try out, etc?

Seems like there are indicators for:
    1. Area in the city the apartment is located
    2. Descriptions of how close the apartment is to certain amenities
    3. The population in terms of gender, workforce, youth, and elder residents
    4. How far away schools are from the apartment
    5. Access to public transportation
   
Additionally, will the time of year do anything for the model? Should there be a "month" indicator?

## Demographics features:
Let's make a youth percent and an elder percent, along with a population density and preschool-aged children/#of schools. 

In [44]:
#Population Density (will be the same throughout each SubArea)
train_clean["pop_density"] = train_clean["raion_popul"] / train_clean["area_m"].astype("float")
test_clean["pop_density"] = test_clean["raion_popul"] / test_clean["area_m"].astype("float")

In [45]:
#Ratio of elder population (will be the same throughout each SubArea)
train_clean["elder_ratio"] = train_clean["ekder_all"] / (train_clean["young_all"] + train_clean["work_all"] + train_clean["ekder_all"]).astype("float")
test_clean["elder_ratio"] = test_clean["ekder_all"] / (test_clean["young_all"] + test_clean["work_all"] + test_clean["ekder_all"]).astype("float")

In [46]:
#Ratio of under 18 population (will be the same throughout each SubArea)
train_clean["youth_ratio"] = train_clean["young_all"] / (train_clean["young_all"] + train_clean["work_all"] + train_clean["ekder_all"]).astype("float")
test_clean["youth_ratio"] = test_clean["young_all"] / (test_clean["young_all"] + test_clean["work_all"] + test_clean["ekder_all"]).astype("float")

In [47]:
#Ratio of number of preschool aged children to number of preschools (will be the same throughout each SubArea)
#train_clean["preschool_ratio"] = train_clean["children_preschool"] / train_clean["preschool_education_centers_raion"].astype("float")
#test_clean["preschool_ratio"] = test_clean["children_preschool"] / test_clean["preschool_education_centers_raion"].astype("float")

#this doesn't look like it worked...

In [52]:
test_clean.describe()

,id,full_sq,floor,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,...,sd_eurrub_yearly,sd_brent_yearly,sd_micex_yearly,sd_mortgage_yearly,sd_rent_yearly,okurg_district,sq_metr,pop_density,elder_ratio,youth_ratio
count,7627.000000,7627.000000,7627.000000,7627.000000,7627.000000,7.627000e+03,7627.000000,7627.000000,7627.000000,7627.000000,...,7627.000000,7627.000000,7627.000000,7627.000000,7627.000000,7627.000000,7627.000000,7627.000000,7627.000000,7627.000000
mean,34302.541366,53.659692,7.628163,0.352170,77.818408,1.771604e+07,87568.730694,0.224886,0.122611,5358.619247,...,6.041245,7.232928,81.359461,0.485145,29.839348,5.371312,53.649748,0.009965,0.227625,0.134636
std,2211.370824,20.077214,5.073153,0.486653,40.163254,2.215256e+07,56422.797102,0.177158,0.126425,3776.542208,...,0.957910,0.776471,9.687323,0.224050,2.152956,3.224945,19.722009,0.006828,0.035683,0.020161
min,30474.000000,0.000000,1.000000,0.000000,0.000000,2.081628e+06,2546.000000,0.001879,0.000000,175.000000,...,5.119272,6.485588,71.295861,0.269500,27.602766,0.000000,10.000000,0.000015,0.114854,0.078007
25%,32387.500000,39.200000,3.000000,0.000000,44.000000,7.307411e+06,37502.000000,0.068202,0.012339,2448.000000,...,5.119272,6.485588,71.295861,0.269500,27.602766,3.000000,39.200000,0.004109,0.215179,0.123241
50%,34300.000000,50.300000,7.000000,0.000000,82.000000,1.004686e+07,85956.000000,0.157332,0.072158,4992.000000,...,5.119272,6.485588,90.683353,0.269500,31.911534,5.000000,50.300000,0.010315,0.226332,0.136339
75%,36219.500000,63.200000,11.000000,1.000000,107.000000,1.788191e+07,118945.000000,0.349935,0.195781,7125.000000,...,7.036363,8.039559,90.683353,0.717897,31.911534,8.000000,63.235000,0.014549,0.255141,0.143464
max,38135.000000,403.000000,41.000000,2.000000,145.000000,2.060718e+08,247469.000000,0.852923,0.521867,19223.000000,...,7.036363,8.039559,90.683353,0.717897,31.911534,12.000000,285.800000,0.028298,0.320316,0.260695


## Select out a few of the variables:
Make a new dataframe with only a select few variables to try out regression with

In [54]:
features = ['id',
 'timestamp',
 'floor',
 'product_type',
 'sub_area',
 'metro_min_walk',
 'kindergarten_km',
 'park_km',
 'kremlin_km',
 'oil_chemistry_km',
 'nuclear_reactor_km',
 'big_market_km',
 'market_shop_km',
 'detention_facility_km',
 'public_healthcare_km',
 'university_km',
 'workplaces_km',
 'preschool_km',
 'big_church_km',
 'oil_urals',
 'cpi',
 'ppi',
 'eurrub',
 'brent',
 'average_provision_of_build_contract_moscow',
 'micex',
 'mortgage_rate',
 'rent_price_4+room_bus',
 'sd_oil_yearly',
 'sd_cpi_yearly',
 'sd_ppi_yearly',
 'sd_eurrub_yearly',
 'sd_brent_yearly',
 'sd_micex_yearly',
 'sd_mortgage_yearly',
 'sd_rent_yearly',
 'okurg_district',
 'sq_metr',
 'pop_density',
 'elder_ratio',
 'youth_ratio',
 'price_doc']

train_trial1 = train_clean[features]
test_trial1 = test_clean[features[:-1]]

In [60]:
train_trial1.shape

(30258, 42)

In [71]:
train_trial1.to_csv('trial_brandy.csv', index = False)

In [72]:
test_trial1.to_csv('test_brandy.csv', index = False)

In [63]:
from sklearn import naive_bayes

In [69]:
## separate the predictors and response in the training data set
x = np.array(train_trial1.iloc[:, 2:41])
y = np.ravel(train_trial1.iloc[:, 41:42])

x
y

array([ 5850000.,  6000000.,  5700000., ...,  6750554.,  5600000.,
        5000000.])

In [70]:
#mnb = naive_bayes.MultinomialNB()
#mnb.fit(x, y)
#print("The score of multinomial naive bayes is: %.4f" %mnb.score(x, y))

In [ ]:
X = train2.drop(["id", "price_doc"], axis = 1)

Y = train2["price_doc"]

model = sm.OLS(Y, X)
X = sm.add_constant(X)
results = model.fit()
print(results.summary())